this notebook include functions to calculate pair distances and sort accordingly

# imports

In [ ]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr,fisher_exact
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist
from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from SampleLists.SampleFileFunctions import *

import os
from Utils import cacheOnDisk
from queue.qp import qp,fakeqp
from addloglevels import sethandlers

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

In [1]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate


'12052018'

# prepare an example R table:

In [ ]:
# load top 250x250 from PNP515_ss15000_rep2:

datasetFolder='%s/TCR_real_data/SubSampled15000data_rep2' %MyPath
datasetName='PNP515_ss15000_rep2'

realRtableFile='%s/TCR_mb_results/permCorrTest_NoneNone250250_stdNone/real_result_df' %datasetFolder
Rtable250250=pd.read_pickle(realRtableFile)

    the function cutSeqSpeciesRtable was developed here and copied to TCR_microbiome_interactions_functions

In [ ]:
# cut to top 100x100
Rtable=Rtable250250
topNseqs=251
topNspecies=260

Rtable100100,topNseqs,topNspecies=cutSeqSpeciesRtable(Rtable,topNseqs,topNspecies)

# develop function to investigate pair distance:

    the function calc_dist_Rtable was developed here and copied to TCR_microbiome_interactions_functions

# check species distances according to correlations with sequences:

## try different metrics:

In [ ]:
pd.set_option('display.width', 1000)
distMetricList=['euclidean','sqeuclidean','canberra']


Rtable=Rtable100100
indexCol='species'
columnsCol='seq'
valueCol='r_real'

for metric in distMetricList:
    print metric
    sortedDistMat=calc_dist_Rtable(Rtable,indexCol,columnsCol,valueCol,metric)
    print sortedDistMat.head(12)

!!! conclusion: 'euclidean' and 'sqeuclidean' are identical. no significant difference between them and 'canberra' in this case. will continue with euclidean. 

## score pairs:

In [ ]:
sortedDistMat.head()

In [ ]:
#score pairs according to being same genus or not:

sortedDistMat['sameGenus']=np.where(sortedDistMat['sample1'].str.split('_').str[0]==sortedDistMat['sample2'].str.split('_').str[0],1,0)
sortedDistMat.head(10)

## compare distances between two scoring groups (binary scoring):

### define the ploting and comparison function:

    the function plot_distribution_for_binary_scoring was developed here and copied to TCR_microbiome_interactions_functions

### run for specific distMat:

In [ ]:
# the one generated in former steps:

binaryScoreCol='sameGenus'
sortedDistMat=sortedDistMat
plotType='density'

fig,ax=plt.subplots()
ax=plot_distribution_for_binary_scoring(ax,sortedDistMat,binaryScoreCol,plotType)
# ax=plot_density_for_binary_scoring(ax,sortedDistMat,binaryScoreCol)
plt.show()

## run for different data set and top pairs

### use histograms:

In [ ]:
datasetFolderList=['%s/TCR_real_data/SubSampled15000data_rep2' %MyPath,
                   '%s/TCR_real_data/SubSampled15000data_rep1' %MyPath,
                  '%s/TCR_real_data/SubSampled9000data_rep2' %MyPath]
datasetNameList=['PNP515_ss15000_rep2','PNP515_ss15000_rep1','PNP515_ss9000_rep2']

topNlist=[20,100,250,400]
metric='euclidean'
plotType='hist'

binaryScoreCol='sameGenus'
sortedDistMat=sortedDistMat

hsize=4*len(datasetNameList)
vsize=3*len(topNlist)
fig, axes=plt.subplots(nrows=len(topNlist), ncols=len(datasetFolderList),figsize=(hsize,vsize),sharey=True)


count=0
sumDF=pd.DataFrame()
    
for i in range(len(datasetFolderList)):
    datasetFolder=datasetFolderList[i]
    datasetName=datasetNameList[i]
    
    #load  r table:
    if i==2:
        realRtableFile='%s/TCR_mb_results/permCorrTest_NoneNone250250_stdNone/real_result_df' %datasetFolder
    else:
        realRtableFile='%s/TCR_mb_results/permCorrTest_NoneNone400400_stdNone/real_result_df' %datasetFolder
    Rtable=pd.read_pickle(realRtableFile)
    print Rtable.shape
    
    for j in range(len(topNlist)):
        topN=topNlist[j]
        
        print count,datasetName, topN
        
        print 'cutting r table to include top %s sequence and species' %topN
        RtableCut,topNseqsNew,topNspeciesNew=cutSeqSpeciesRtable(Rtable,topNseqs=topN,topNspecies=topN)
        
        ax=axes[j,i]
        #GENERATE DISTANCE MATRIX:
        sortedDistMat=calc_dist_Rtable(RtableCut,indexCol='species',columnsCol='seq',valueCol='r_real',distMetric=metric)
        
        #SCORE MATRIX ACCORDING TO SAME GENUS:
        binaryScoreCol='sameGenus'
        sortedDistMat[binaryScoreCol]=np.where(sortedDistMat['sample1'].str.split('_').str[0]==sortedDistMat['sample2'].str.split('_').str[0],1,0)
        
        #plot histogram for specific Rtable
        print 'plotting histograms...'
        ax1,mean1,mean2,ks_p_cohort1_cohort2,t_p_cohort1_cohort2,normDif=plot_distribution_for_binary_scoring\
        (ax,sortedDistMat,binaryScoreCol,plotType=plotType)
        if i==1:
            ax1.set_title('%s\ntop%sx%s\nOnly Nextera samples' %(datasetName,topNseqsNew,topNspeciesNew))
        else:
            ax1.set_title('%s\ntop%sx%s' %(datasetName,topNseqsNew,topNspeciesNew))
#         ax.set_yscale('log')
        
        
        sumDF.loc[count,'metric']=metric
        sumDF.loc[count,'datasetName']=datasetName
        sumDF.loc[count,'topNseqs']=topNseqsNew
        sumDF.loc[count,'topNspecies']=topNspeciesNew
        sumDF.loc[count,'mean: %s=0' %binaryScoreCol]=mean1
        sumDF.loc[count,'mean: %s=1' %binaryScoreCol]=mean2
        sumDF.loc[count,'normDif']=normDif
        sumDF.loc[count,'p_KS']=ks_p_cohort1_cohort2
        sumDF.loc[count,'t_p']=t_p_cohort1_cohort2
        sumDF.loc[count,'normDif']=normDif
        
        if count==len(datasetFolderList)*len(topNlist)-1:
            ax.legend(bbox_to_anchor=(0.1, -0.2), loc='upper left', borderaxespad=0.,fontsize=16)
        else:
            ax.legend().set_visible(False)
        count=count+1

print 'saving files...'
fig.subplots_adjust(left=0.1, bottom=None, right=0.99, top=0.95,
    wspace=0.2, hspace=0.4)



figFolder='%s/TCR_real_data/TCR_mb_results/DATA_SUMMARY_ALL_DATASETS/speciesClusterBySequence' %MyPath
if not isdir(figFolder):
    makedirs(figFolder)
figFile='%s/same_notSame_genus_distance_hists_%s' %(figFolder,metric)
fig.savefig(figFile,dpi=200)

dfFile='%s/same_notSame_genus_distance_summary_%s.xlsx' %(figFolder,metric)
sumDF.to_excel(dfFile)
        


In [ ]:
plt.show()

### use density plots:

In [ ]:
datasetFolderList=['%s/TCR_real_data/SubSampled15000data_rep2' %MyPath,
                   '%s/TCR_real_data/SubSampled15000data_rep1' %MyPath,
                  '%s/TCR_real_data/SubSampled9000data_rep2' %MyPath]
datasetNameList=['PNP515_ss15000_rep2','PNP515_ss15000_rep1','PNP515_ss9000_rep2']

topNlist=[20,100,250,400]
metric='euclidean'
plotType='density'

hsize=4*len(datasetNameList)
vsize=4*len(topNlist)
fig, axes=plt.subplots(nrows=len(topNlist), ncols=len(datasetFolderList),figsize=(hsize,vsize),sharey=True)


count=0
sumDF=pd.DataFrame()
    
for i in range(len(datasetFolderList)):
    datasetFolder=datasetFolderList[i]
    datasetName=datasetNameList[i]
    
    #load  r table:
    if i==2:
        realRtableFile='%s/TCR_mb_results/permCorrTest_NoneNone250250_stdNone/real_result_df' %datasetFolder
    else:
        realRtableFile='%s/TCR_mb_results/permCorrTest_NoneNone400400_stdNone/real_result_df' %datasetFolder
    Rtable=pd.read_pickle(realRtableFile)
    print Rtable.shape
    
    for j in range(len(topNlist)):
        topN=topNlist[j]
        
        print count,datasetName, topN
        
        print 'cutting r table to include top %s sequence and species' %topN
        RtableCut,topNseqsNew,topNspeciesNew=cutSeqSpeciesRtable(Rtable,topNseqs=topN,topNspecies=topN)
        
        ax=axes[j,i]
        #GENERATE DISTANCE MATRIX:
        sortedDistMat=calc_dist_Rtable(RtableCut,indexCol='species',columnsCol='seq',valueCol='r_real',distMetric=metric)
        
        #SCORE MATRIX ACCORDING TO SAME GENUS:
        binaryScoreCol='sameGenus'
        sortedDistMat[binaryScoreCol]=np.where(sortedDistMat['sample1'].str.split('_').str[0]==sortedDistMat['sample2'].str.split('_').str[0],1,0)
        
        #plot histogram for specific Rtable
        print 'plotting histograms...'
        ax1,mean1,mean2,ks_p_cohort1_cohort2,t_p_cohort1_cohort2,normDif=plot_distribution_for_binary_scoring\
        (ax,sortedDistMat,binaryScoreCol,plotType)
        if i==1:
            ax1.set_title('%s\ntop%sx%s\nOnly Nextera samples' %(datasetName,topNseqsNew,topNspeciesNew))
        else:
            ax1.set_title('%s\ntop%sx%s' %(datasetName,topNseqsNew,topNspeciesNew))
#         ax.set_yscale('log')
        
        
        sumDF.loc[count,'metric']=metric
        sumDF.loc[count,'datasetName']=datasetName
        sumDF.loc[count,'topNseqs']=topNseqsNew
        sumDF.loc[count,'topNspecies']=topNspeciesNew
        sumDF.loc[count,'mean: %s=0' %binaryScoreCol]=mean1
        sumDF.loc[count,'mean: %s=1' %binaryScoreCol]=mean2
        sumDF.loc[count,'normDif']=normDif
        sumDF.loc[count,'p_KS']=ks_p_cohort1_cohort2
        sumDF.loc[count,'t_p']=t_p_cohort1_cohort2      
        
        if count==len(datasetFolderList)*len(topNlist)-1:
            ax1.legend(bbox_to_anchor=(0.1, -0.2), loc='upper left', borderaxespad=0.,fontsize=16)
        else:
            ax1.legend().set_visible(False)
        count=count+1

print 'saving files...'
fig.subplots_adjust(left=0.1, bottom=None, right=0.99, top=0.95,
    wspace=0.15, hspace=0.5)



figFolder='%s/TCR_real_data/TCR_mb_results/DATA_SUMMARY_ALL_DATASETS/speciesClusterBySequence' %MyPath
if not isdir(figFolder):
    makedirs(figFolder)
figFile='%s/same_notSame_genus_distance_%s_%s' %(figFolder,plotType,metric)
fig.savefig(figFile,dpi=200)

dfFile='%s/same_notSame_genus_distance_summary_%s.xlsx' %(figFolder,metric)
sumDF.to_excel(dfFile)
        


In [ ]:
plt.show()

### calculate segregation for many different numbers of species and sequences and datasets, without plotting

In [ ]:
datasetFolderList=['%s/TCR_real_data/SubSampled15000data_rep2' %MyPath,
                   '%s/TCR_real_data/SubSampled15000data_rep1' %MyPath,
                  '%s/TCR_real_data/SubSampled9000data_rep2' %MyPath]
datasetNameList=['PNP515_ss15000_rep2','PNP515_ss15000_rep1','PNP515_ss9000_rep2']
topNseqsList=[20,50,100,250,400]
topNspeciesList=[20,50,100,250,400]

metric='euclidean'
plotType='density'

hsize=4*len(datasetNameList)
vsize=4*len(topNlist)
fig, axes=plt.subplots(nrows=len(datasetFolderList)*len(topNseqsList)*len(topNspeciesList),ncols=1,
                       figsize=(hsize,vsize),sharey=True)


count=0
sumDF=pd.DataFrame()
    
for i in range(len(datasetFolderList)):
    datasetFolder=datasetFolderList[i]
    datasetName=datasetNameList[i]
    
    #load  r table:
    if i==2:
        realRtableFile='%s/TCR_mb_results/permCorrTest_NoneNone250250_stdNone/real_result_df' %datasetFolder
    else:
        realRtableFile='%s/TCR_mb_results/permCorrTest_NoneNone400400_stdNone/real_result_df' %datasetFolder
    Rtable=pd.read_pickle(realRtableFile)
    print Rtable.shape
    
    for j in range(len(topNseqsList)):
        topNseqs=topNseqsList[j]
        
        for k in range(len(topNspeciesList)):
            topNspecies=topNspeciesList[k]

            print count,datasetName, topNseqs,topNspecies

            print 'cutting r table to include top %s sequence and %s species' %(topNseqs,topNspecies)
            RtableCut,topNseqsNew,topNspeciesNew=cutSeqSpeciesRtable(Rtable,topNseqs,topNspecies)

            ax=axes.flatten()[count]
            #GENERATE DISTANCE MATRIX:
            sortedDistMat=calc_dist_Rtable(RtableCut,indexCol='species',columnsCol='seq',valueCol='r_real',distMetric=metric)

            #SCORE MATRIX ACCORDING TO SAME GENUS:
            binaryScoreCol='sameGenus'
            sortedDistMat[binaryScoreCol]=np.where(sortedDistMat['sample1'].str.split('_').str[0]==sortedDistMat['sample2'].str.split('_').str[0],1,0)

            #plot histogram for specific Rtable
            print 'plotting histograms...'
            ax1,mean1,mean2,ks_p_cohort1_cohort2,t_p_cohort1_cohort2,normDif=plot_distribution_for_binary_scoring\
            (ax,sortedDistMat,binaryScoreCol,plotType)
            if i==1:
                ax1.set_title('%s\ntop%sx%s\nOnly Nextera samples' %(datasetName,topNseqsNew,topNspeciesNew))
            else:
                ax1.set_title('%s\ntop%sx%s' %(datasetName,topNseqsNew,topNspeciesNew))
    #         ax.set_yscale('log')


            sumDF.loc[count,'metric']=metric
            sumDF.loc[count,'datasetName']=datasetName
            sumDF.loc[count,'topNseqs']=topNseqsNew
            sumDF.loc[count,'topNspecies']=topNspeciesNew
            sumDF.loc[count,'mean: %s=0' %binaryScoreCol]=mean1
            sumDF.loc[count,'mean: %s=1' %binaryScoreCol]=mean2
            sumDF.loc[count,'normDif']=normDif
            sumDF.loc[count,'p_KS']=ks_p_cohort1_cohort2
            sumDF.loc[count,'t_p']=t_p_cohort1_cohort2      

            if count==len(datasetFolderList)*len(topNlist)-1:
                ax1.legend(bbox_to_anchor=(0.1, -0.2), loc='upper left', borderaxespad=0.,fontsize=16)
            else:
                ax1.legend().set_visible(False)
            count=count+1

print 'saving files...'
fig.subplots_adjust(left=0.1, bottom=None, right=0.99, top=0.95,
    wspace=0.15, hspace=0.5)



# figFolder='%s/TCR_real_data/TCR_mb_results/DATA_SUMMARY_ALL_DATASETS/speciesClusterBySequence' %MyPath
# if not isdir(figFolder):
#     makedirs(figFolder)
# figFile='%s/same_notSame_genus_distance_%s_%s' %(figFolder,plotType,metric)
# fig.savefig(figFile,dpi=200)

dfFile='%s/same_notSame_genus_distance_summary_%s_hugh.xlsx' %(figFolder,metric)
sumDF.to_excel(dfFile)
        


!!! conclusion: Use top 250x250 from any dataset, or top 20 from rep1-only nextera set, for further analysis!

# cluster species by correlations with sequences:

In [ ]:
#######

##copy back to .py and delete from notebook!

#####


def gen_heatmap_and_clustermap(Rtable, RtableName, figFolder,indexCol,columnsCol,valueCol,metric='euclidean',
                               method='average',sortMethod='max',cmapName='hsv'):
                               
#                                figsizeX, figsizeY, fontsizeX, fontsizeY, fontSizeLabel, sortMethod):
    
    nInd= Rtable[indexCol].nunique()  
    nCol=Rtable[columnsCol].nunique()
    if nInd<100:
        figsizeY=float(nInd)/2
    else:
        figsizeY=float(nInd)/5
    if nCol<100:
        figsizeX=float(nCol)/2
    else:
        figsizeX=float(nCol)/5
    
       
    print 'Rtable shape is %s_%s' % (Rtable.shape[0], Rtable.shape[1])
    print 'number of unique %s=%s' % (indexCol,Rtable[indexCol].nunique())
    print 'number of unique %s=%s' % (columnsCol,Rtable[columnsCol].nunique())  
        
    # generate pivot table:
    print 'generating pivot table...'
    pivot = Rtable.pivot(index=indexCol, columns=columnsCol, values=valueCol)
    print pivot.shape
    if sortMethod == 'max':
        pivot[sortMethod] = pivot.max(axis=1)
        pivot.loc[sortMethod, :] = pivot.max(axis=0)
    elif sortMethod == 'mean':
        pivot[sortMethod] = pivot.mean(axis=1)
        pivot.loc[sortMethod, :] = pivot.mean(axis=0)
    pivot = pivot.sort_values(by=sortMethod, axis=1)
    pivot = pivot.sort_values(by=sortMethod, axis=0)
    pivot = pivot.drop(sortMethod, axis=1)
    pivot = pivot.drop(sortMethod, axis=0)
    
    
    if indexCol=='species':
        # generate genus color list:
        uniqueSpecies = pivot.index.unique()
        genus = [x.split('_')[0] for x in uniqueSpecies ]
        genus2 = list(set(genus))
        genus2n = len(genus2)
        if genus2n < 10:
            fact = 30
        elif genus2n < 20:
            fact = 20
        elif genus2n < 50:
            fact = 10
        else:
            fact = 10 
        species_genus_df = pd.DataFrame({'species':uniqueSpecies, 'genus':genus})
        genus2color = []
        for n, g in enumerate(genus2):
            color = plt.cm.get_cmap(cmapName)(fact * n)
            genus2color.append(color)
        genus2colorDF = pd.DataFrame({'genus':genus2, 'color':genus2color})
        speciesGenusColorDF = pd.merge(species_genus_df, genus2colorDF, how='left', left_on='genus', right_on='genus')
        cluster_colors = list(speciesGenusColorDF['color'])
    
    # generate heatmap:
    print 'generate heatmap...'
    heatMapFig, ax = plt.subplots(figsize=(figsizeX, figsizeY))  # Sample figsize in inches
    sns.heatmap(pivot, ax=ax)
#     ax.yaxis.label.set_size(fontSizeLabel)
#     ax.tick_params(axis='y', labelsize=fontsizeY)
#     ax.xaxis.label.set_size(fontSizeLabel)
#     ax.tick_params(axis='x', labelsize=fontsizeX)
    heatMapName = '%s_heatMap_%sBy%s_sortBy%s' % (RtableName, indexCol,columnsCol, sortMethod)
    heatMapFile='%s/%s' %(figFolder,heatMapName)
#     heatMapFig.savefig(heatMapFile, dpi=200)
    
    # generate clustermap
    print 'generate clustermap...'
    if indexCol=='species': 
        cg = sns.clustermap(pivot, figsize=(figsizeX, figsizeY), col_cluster=False, row_colors=cluster_colors,metric=metric,
                            method=method)
    else:
        cg = sns.clustermap(pivot, figsize=(figsizeX, figsizeY), col_cluster=False,metric=metric,method=method)
    
    plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
    plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels())
    plt.title(RtableName)
    clusterMapName = '%s_clusterMap_%s_%s' % (RtableName, metric,method)
    clusterMapFile='%s/%s' %(figFolder,clusterMapName)
    cg.savefig(clusterMapFile, dpi=80)
    
    print 'done'
    
    return heatMapFig, cg

In [ ]:
# load top 250x250 from PNP515_ss15000_rep2:

datasetFolder='%s/TCR_real_data/SubSampled9000data_rep2' %MyPath
datasetName='PNP515_ss9000_rep2'
realRtableFile='%s/TCR_mb_results/permCorrTest_NoneNone250250_stdNone/real_result_df' %datasetFolder
Rtable400400=pd.read_pickle(realRtableFile)

# cut:
Rtable=Rtable250250
topNseqs=100
topNspecies=20

RtableCut,topNseqs,topNspecies=cutSeqSpeciesRtable(Rtable,topNseqs,topNspecies)


Rtable=RtableCut
RtableName='%s_%sspeciesx%sseqs' %(datasetName,topNspecies,topNseqs)
figFolder='%s/TCR_real_data/TCR_mb_results/DATA_SUMMARY_ALL_DATASETS/speciesClusterBySequence/clusterMap' %MyPath
if not isdir(figFolder):
    makedirs(figFolder)
indexCol='species'
columnsCol='seq'
valueCol='r_real'
metric='euclidean'
method='complete'
sortMethod='max'
cmapName='gist_ncar'


heatMapFig, cg=heatMapFig, cg=gen_heatmap_and_clustermap(Rtable, RtableName, figFolder,indexCol,columnsCol,
                            valueCol,metric, method,sortMethod,cmapName)

In [ ]:
plt.show()

# check sequence distances according to correlations with species:

## generate distance table:

In [ ]:
#generate distance table for sequences:
pd.set_option('display.width', 1000)
distMetricList=['euclidean']


Rtable=Rtable100100
indexCol='seq'
columnsCol='species'
valueCol='r_real'

for metric in distMetricList:
    print metric
    sortedDistMat=calc_dist_Rtable(Rtable,indexCol,columnsCol,valueCol,metric)
    print sortedDistMat.head(12)

## merge with identity:

In [ ]:


file1='%s/TCR CDR3 sequence databases/CDR3identityTable_23042018_processed.xlsx' %MyPath
CDR3identityTable=pd.read_excel(file1)
CDR3identityTable.head()

In [ ]:
identityColumn='Pathology_McPAS'
# identityColumn='Category_McPAS'

sortedDistMat_wIdentity=pd.merge(sortedDistMat,pd.DataFrame(CDR3identityTable[identityColumn]),
                                 how='left',left_on='sample1', right_index=True)
sortedDistMat_wIdentity=sortedDistMat_wIdentity.rename(columns={identityColumn:'sample1_%s' %identityColumn})
sortedDistMat_wIdentity=pd.merge(sortedDistMat_wIdentity,pd.DataFrame(CDR3identityTable[identityColumn]),
                                 how='left',left_on='sample2', right_index=True)
sortedDistMat_wIdentity=sortedDistMat_wIdentity.rename(columns={identityColumn:'sample2_%s' %identityColumn})
sortedDistMat_wIdentity.head(10)

## score by identity

In [ ]:
#function that checks whether an item in list 1 is contained within list 2 and returns 1 if True
def iden_score(list1,list2):
    answer=0
    for term in list1:
        if term in list2:
            answer=1
    return answer
        
        

In [ ]:
#score the sequence pairs: if there are common identities between the two sequences, 
#they get score of 1, if not, they get score of 0


scored=sortedDistMat_wIdentity.dropna()
scored['sample1_%s' %identityColumn]=scored['sample1_%s' %identityColumn].str.replace(' ','')
scored['sample2_%s' %identityColumn]=scored['sample2_%s' %identityColumn].str.replace(' ','')
scored['sample1_%s_list'  %identityColumn]=scored['sample1_%s' %identityColumn].str.split(',')
scored['sample2_%s_list'  %identityColumn]=scored['sample2_%s' %identityColumn].str.split(',')
scored['sameIdentity']=scored.apply(lambda x:\
iden_score(x['sample1_%s_list'  %identityColumn],x['sample2_%s_list'  %identityColumn]),axis=1)

scored.head(10)

In [ ]:
sortedDistMat_wIdentity_scored=pd.merge(sortedDistMat_wIdentity,scored[['sample1','sample2','sameIdentity']],
                                       how='left',left_on=['sample1','sample2'],right_on=['sample1','sample2'])
# sortedDistMat_wIdentity_scored['sameIdentity']=sortedDistMat_wIdentity_scored['sameIdentity'].fillna(0)
sortedDistMat_wIdentity_scored[sortedDistMat_wIdentity_scored['sameIdentity']==1].sort_index().head()

In [ ]:
sortedDistMat_wIdentity_scored[sortedDistMat_wIdentity_scored['sameIdentity']==0].sort_index().head()

In [ ]:
sortedDistMat_wIdentity_scored['sameIdentity'].value_counts(dropna=False)

### compare distances between scores:

In [ ]:
# the one generated in former steps:

binaryScoreCol='sameIdentity'
sortedDistMat=sortedDistMat_wIdentity_scored
plotType='density'

fig,ax=plt.subplots()
ax=plot_distribution_for_binary_scoring(ax,sortedDistMat,binaryScoreCol,plotType)
# ax=plot_density_for_binary_scoring(ax,sortedDistMat,binaryScoreCol)
plt.show()

In [ ]:
# the one generated in former steps:

binaryScoreCol='sameIdentity'
sortedDistMat=sortedDistMat_wIdentity_scored
plotType='hist'

fig,ax=plt.subplots()
ax=plot_distribution_for_binary_scoring(ax,sortedDistMat,binaryScoreCol,plotType)
# ax=plot_density_for_binary_scoring(ax,sortedDistMat,binaryScoreCol)
plt.show()

### calculate segregation for many different numbers of species and sequences and datasets, without plotting

In [ ]:
file1='%s/TCR CDR3 sequence databases/CDR3identityTable_23042018_processed.xlsx' %MyPath
CDR3identityTable=pd.read_excel(file1)


datasetFolderList=['%s/TCR_real_data/SubSampled15000data_rep2' %MyPath,
                   '%s/TCR_real_data/SubSampled15000data_rep1' %MyPath,
                  '%s/TCR_real_data/SubSampled9000data_rep2' %MyPath]
datasetNameList=['PNP515_ss15000_rep2','PNP515_ss15000_rep1','PNP515_ss9000_rep2']
topNseqsList=[20,50,100,250,400]
topNspeciesList=[20,50,100,250,400]

metric='euclidean'
plotType='density'
# identityColumn='Pathology_McPAS' #('Pathology_McPAS' or 'Category_McPAS')
# identityColumn='Category_McPAS'
identityColumn='Epitope species_VDJDB'

hsize=4*len(datasetNameList)
vsize=4*len(topNlist)
fig, axes=plt.subplots(nrows=len(datasetFolderList)*len(topNseqsList)*len(topNspeciesList),ncols=1,
                       figsize=(hsize,vsize),sharey=True)


count=0
sumDF=pd.DataFrame()
    
for i in range(len(datasetFolderList)):
    datasetFolder=datasetFolderList[i]
    datasetName=datasetNameList[i]
    
    #load  r table:
    if i==2:
        realRtableFile='%s/TCR_mb_results/permCorrTest_NoneNone250250_stdNone/real_result_df' %datasetFolder
    else:
        realRtableFile='%s/TCR_mb_results/permCorrTest_NoneNone400400_stdNone/real_result_df' %datasetFolder
    Rtable=pd.read_pickle(realRtableFile)
    print Rtable.shape
    
    for j in range(len(topNseqsList)):
        topNseqs=topNseqsList[j]
        
        for k in range(len(topNspeciesList)):
            topNspecies=topNspeciesList[k]

            print count,datasetName, topNseqs,topNspecies

            print 'cutting r table to include top %s sequence and %s species' %(topNseqs,topNspecies)
            RtableCut,topNseqsNew,topNspeciesNew=cutSeqSpeciesRtable(Rtable,topNseqs,topNspecies)

            ax=axes.flatten()[count]
            #GENERATE DISTANCE MATRIX:
            sortedDistMat=calc_dist_Rtable(RtableCut,indexCol='seq',columnsCol='species',valueCol='r_real',distMetric=metric)

            #SCORE MATRIX ACCORDING TO SAME IDENTITY:
            
            ##add identity:
            sortedDistMat_wIdentity=pd.merge(sortedDistMat,pd.DataFrame(CDR3identityTable[identityColumn]),
                                 how='left',left_on='sample1', right_index=True)
            sortedDistMat_wIdentity=sortedDistMat_wIdentity.rename(columns={identityColumn:'sample1_%s' %identityColumn})
            sortedDistMat_wIdentity=pd.merge(sortedDistMat_wIdentity,pd.DataFrame(CDR3identityTable[identityColumn]),
                                             how='left',left_on='sample2', right_index=True)
            sortedDistMat_wIdentity=sortedDistMat_wIdentity.rename(columns={identityColumn:'sample2_%s' %identityColumn})
            
            #score the sequence pairs: if there are common identities between the two sequences, 
            #they get score of 1, if not, they get score of 0


            scored=sortedDistMat_wIdentity.dropna()
            scored['sample1_%s' %identityColumn]=scored['sample1_%s' %identityColumn].str.replace(' ','')
            scored['sample2_%s' %identityColumn]=scored['sample2_%s' %identityColumn].str.replace(' ','')
            scored['sample1_%s_list'  %identityColumn]=scored['sample1_%s' %identityColumn].str.split(',')
            scored['sample2_%s_list'  %identityColumn]=scored['sample2_%s' %identityColumn].str.split(',')
            scored['sameIdentity']=scored.apply(lambda x:\
            iden_score(x['sample1_%s_list'  %identityColumn],x['sample2_%s_list'  %identityColumn]),axis=1)
            sortedDistMat_wIdentity_scored=pd.merge(sortedDistMat_wIdentity,scored[['sample1','sample2','sameIdentity']],
                                       how='left',left_on=['sample1','sample2'],right_on=['sample1','sample2'])
#             sortedDistMat_wIdentity_scored['sameIdentity']=sortedDistMat_wIdentity_scored['sameIdentity'].fillna(0)
            
            
            #plot histogram for specific Rtable
            print 'plotting histograms...'
            ax1,mean1,mean2,ks_p_cohort1_cohort2,t_p_cohort1_cohort2,normDif=plot_distribution_for_binary_scoring\
            (ax,sortedDistMat_wIdentity_scored,binaryScoreCol,plotType)
            if i==1:
                ax1.set_title('%s\ntop%sx%s\nOnly Nextera samples' %(datasetName,topNseqsNew,topNspeciesNew))
            else:
                ax1.set_title('%s\ntop%sx%s' %(datasetName,topNseqsNew,topNspeciesNew))
    #         ax.set_yscale('log')


            sumDF.loc[count,'metric']=metric
            sumDF.loc[count,'datasetName']=datasetName
            sumDF.loc[count,'topNseqs']=topNseqsNew
            sumDF.loc[count,'topNspecies']=topNspeciesNew
            sumDF.loc[count,'mean: %s=0' %binaryScoreCol]=mean1
            sumDF.loc[count,'mean: %s=1' %binaryScoreCol]=mean2
            sumDF.loc[count,'normDif']=normDif
            sumDF.loc[count,'p_KS']=ks_p_cohort1_cohort2
            sumDF.loc[count,'t_p']=t_p_cohort1_cohort2      

            if count==len(datasetFolderList)*len(topNlist)-1:
                ax1.legend(bbox_to_anchor=(0.1, -0.2), loc='upper left', borderaxespad=0.,fontsize=16)
            else:
                ax1.legend().set_visible(False)
            count=count+1

print 'saving files...'
fig.subplots_adjust(left=0.1, bottom=None, right=0.99, top=0.85,
    wspace=0.15, hspace=0.5)



figFolder='%s/TCR_real_data/TCR_mb_results/DATA_SUMMARY_ALL_DATASETS/seqsClusterBySpecies' %MyPath
if not isdir(figFolder):
    makedirs(figFolder)
# figFile='%s/same_notSame_genus_distance_%s_%s' %(figFolder,plotType,metric)
# fig.savefig(figFile,dpi=200)

dfFile='%s/same_notSame_identity_sequence_distance_summary_%s_hugh_identityColumn%s.xlsx' %(figFolder,metric,identityColumn)
sumDF.to_excel(dfFile)
        


### use plots:

In [ ]:
datasetFolderList=['%s/TCR_real_data/SubSampled15000data_rep2' %MyPath,
                   '%s/TCR_real_data/SubSampled15000data_rep1' %MyPath,
                  '%s/TCR_real_data/SubSampled9000data_rep2' %MyPath]
datasetNameList=['PNP515_ss15000_rep2','PNP515_ss15000_rep1','PNP515_ss9000_rep2']

topNlist=[20,100,250,400]
metric='euclidean'
plotType='density'

binaryScoreCol='sameIdentity'
sortedDistMat=sortedDistMat
metric='euclidean'
plotType='density'
# identityColumn='Pathology_McPAS' #('Pathology_McPAS' or 'Category_McPAS')
# identityColumn='Category_McPAS'
identityColumn='Epitope species_VDJDB'


hsize=4*len(datasetNameList)
vsize=4*len(topNlist)
fig, axes=plt.subplots(nrows=len(topNlist), ncols=len(datasetFolderList),figsize=(hsize,vsize),sharey=True)


count=0
sumDF=pd.DataFrame()
    
for i in range(len(datasetFolderList)):
    datasetFolder=datasetFolderList[i]
    datasetName=datasetNameList[i]
    
    #load  r table:
    if i==2:
        realRtableFile='%s/TCR_mb_results/permCorrTest_NoneNone250250_stdNone/real_result_df' %datasetFolder
    else:
        realRtableFile='%s/TCR_mb_results/permCorrTest_NoneNone400400_stdNone/real_result_df' %datasetFolder
    Rtable=pd.read_pickle(realRtableFile)
    print Rtable.shape
    
    for j in range(len(topNlist)):
        topN=topNlist[j]
        
        print count,datasetName, topN
        
        print 'cutting r table to include top %s sequence and species' %topN
        RtableCut,topNseqsNew,topNspeciesNew=cutSeqSpeciesRtable(Rtable,topNseqs=topN,topNspecies=topN)
        
        ax=axes[j,i]
        #GENERATE DISTANCE MATRIX:
        sortedDistMat=calc_dist_Rtable(RtableCut,indexCol='seq',columnsCol='species',valueCol='r_real',distMetric=metric)

        #SCORE MATRIX ACCORDING TO SAME IDENTITY:

        ##add identity:
        sortedDistMat_wIdentity=pd.merge(sortedDistMat,pd.DataFrame(CDR3identityTable[identityColumn]),
                             how='left',left_on='sample1', right_index=True)
        sortedDistMat_wIdentity=sortedDistMat_wIdentity.rename(columns={identityColumn:'sample1_%s' %identityColumn})
        sortedDistMat_wIdentity=pd.merge(sortedDistMat_wIdentity,pd.DataFrame(CDR3identityTable[identityColumn]),
                                         how='left',left_on='sample2', right_index=True)
        sortedDistMat_wIdentity=sortedDistMat_wIdentity.rename(columns={identityColumn:'sample2_%s' %identityColumn})

        #score the sequence pairs: if there are common identities between the two sequences, 
        #they get score of 1, if not, they get score of 0


        scored=sortedDistMat_wIdentity.dropna()
        scored['sample1_%s' %identityColumn]=scored['sample1_%s' %identityColumn].str.replace(' ','')
        scored['sample2_%s' %identityColumn]=scored['sample2_%s' %identityColumn].str.replace(' ','')
        scored['sample1_%s_list'  %identityColumn]=scored['sample1_%s' %identityColumn].str.split(',')
        scored['sample2_%s_list'  %identityColumn]=scored['sample2_%s' %identityColumn].str.split(',')
        scored[binaryScoreCol]=scored.apply(lambda x:\
        iden_score(x['sample1_%s_list'  %identityColumn],x['sample2_%s_list'  %identityColumn]),axis=1)
        sortedDistMat_wIdentity_scored=pd.merge(sortedDistMat_wIdentity,scored[['sample1','sample2',binaryScoreCol]],
                                   how='left',left_on=['sample1','sample2'],right_on=['sample1','sample2'])
#             sortedDistMat_wIdentity_scored['binaryScoreCol']=sortedDistMat_wIdentity_scored['binaryScoreCol'].fillna(0)

    #plot histogram for specific Rtable
        print 'plotting histograms...'
        
        fig.suptitle('sequence pair distance based on correlation with species\n\
        comparison between same pairs and non-same identity pairs\n\
        identityColumn=%s' %identityColumn)
        
        
        ax1,mean1,mean2,ks_p_cohort1_cohort2,t_p_cohort1_cohort2,normDif=plot_distribution_for_binary_scoring\
        (ax,sortedDistMat_wIdentity_scored,binaryScoreCol,plotType=plotType)
        if i==1:
            ax1.set_title('%s\ntop%sx%s\nOnly Nextera samples' %(datasetName,topNseqsNew,topNspeciesNew))
        else:
            ax1.set_title('%s\ntop%sx%s' %(datasetName,topNseqsNew,topNspeciesNew))
#         ax.set_yscale('log')
        
        
        sumDF.loc[count,'metric']=metric
        sumDF.loc[count,'datasetName']=datasetName
        sumDF.loc[count,'topNseqs']=topNseqsNew
        sumDF.loc[count,'topNspecies']=topNspeciesNew
        sumDF.loc[count,'mean: %s=0' %binaryScoreCol]=mean1
        sumDF.loc[count,'mean: %s=1' %binaryScoreCol]=mean2
        sumDF.loc[count,'normDif']=normDif
        sumDF.loc[count,'p_KS']=ks_p_cohort1_cohort2
        sumDF.loc[count,'t_p']=t_p_cohort1_cohort2
        sumDF.loc[count,'normDif']=normDif
        
        if count==len(datasetFolderList)*len(topNlist)-1:
            ax.legend(bbox_to_anchor=(0.1, -0.2), loc='upper left', borderaxespad=0.,fontsize=16)
        else:
            ax.legend().set_visible(False)
        count=count+1

print 'saving files...'
fig.subplots_adjust(left=0.1, bottom=None, right=0.99, top=0.85,
    wspace=0.2, hspace=0.4)



figFolder='%s/TCR_real_data/TCR_mb_results/DATA_SUMMARY_ALL_DATASETS/seqsClusterBySpecies' %MyPath
if not isdir(figFolder):
    makedirs(figFolder)
figFile='%s/same_notSame_identity_sequence_distance_%s_%s_identityColumn%s' %(figFolder,plotType,metric,identityColumn)
fig.savefig(figFile,dpi=200)

dfFile='%s/same_notSame_identity_sequence_distance_summary_%s_identityColumn%s.xlsx' %(figFolder,metric,identityColumn)
sumDF.to_excel(dfFile)
        


## score by related-to diabetes

In [ ]:
#score the sequence pairs: if there are common identities between the two sequences, 
#they get score of 1, if not, they get score of 0

print identityColumn
scored=sortedDistMat_wIdentity.dropna()
scored['sample1_%s' %identityColumn]=scored['sample1_%s' %identityColumn].str.replace(' ','')
scored['sample2_%s' %identityColumn]=scored['sample2_%s' %identityColumn].str.replace(' ','')
scored['sample1_%s_list'  %identityColumn]=scored['sample1_%s' %identityColumn].str.split(',')
scored['sample2_%s_list'  %identityColumn]=scored['sample2_%s' %identityColumn].str.split(',')
scored['sameIdentity']=scored.apply(lambda x:\
iden_score(x['sample1_%s_list'  %identityColumn],x['sample2_%s_list'  %identityColumn]),axis=1)
scored.head()


In [ ]:
scored['sample1_diabetes']=scored['sample1_Pathology_McPAS_list'].apply(lambda x: 'DiabetesType1' in x)
scored['sample2_diabetes']=scored['sample2_Pathology_McPAS_list'].apply(lambda x: 'DiabetesType1' in x)
scored['combined_diabetes']=np.where(scored['sample1_diabetes']==scored['sample2_diabetes',1,0])
# scored['sample1_diabetes']=np.where(scored['sample1_Pathology_McPAS_list'].str.contains('DiabetesType1'),1,0)
scored.head()

In [ ]:
True+False

In [ ]:
#define all pathologies in the list:
pathologies1=scored['sample1_Pathology_McPAS_list'].tolist()
pathologies2=scored['sample2_Pathology_McPAS_list'].tolist()
pathologies=pathologies1+pathologies2


In [ ]:
#generate a list of unique pathologies:
patholgies_flatten=[]
for a in pathologies:
    for b in a:
        patholgies_flatten.append(b)
uniquePathologies=np.unique(patholgies_flatten)
print len(uniquePathologies)

In [ ]:
for pathology in uniquePathologies:
    print pathology
    scored['sample1_%s' %pathology]=scored['sample1_Pathology_McPAS_list'].apply(lambda x: pathology in x)
    scored['sample2_%s' %pathology]=scored['sample2_Pathology_McPAS_list'].apply(lambda x: pathology in x)
    scored['combined_%s' %pathology]=scored.apply(lambda x: x['sample1_Pathology_McPAS_list']\
                                                  and x['sample2_Pathology_McPAS_list'])
scored.head(20)
    

In [ ]:
l=scored['sample2_Pathology_McPAS_list'].tolist()

In [ ]:
l2=[]
for a in l:
    for b in a:
        l2.append(b)
l2[:20]

In [ ]:
l3=np.unique(l2)
l3

In [ ]:
print len(l3)